In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.llms import ChatGLM

In [2]:
endpoint_url = "http://127.0.0.1:8000/v1/chat/completions"
llm = ChatGLM(endpoint_url=endpoint_url)

In [3]:
embeddings = HuggingFaceEmbeddings(model_name='shibing624/text2vec-base-chinese',model_kwargs={'device':'cuda:0'},encode_kwargs={'normalize_embeddings':False})

db = Chroma(persist_directory='VectorStore', embedding_function=embeddings)

In [4]:
# s = db.similarity_search('卵巢功能')
# s


In [5]:
retriever = db.as_retriever()

In [6]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type = 'stuff',
    retriever=retriever
)

In [7]:
response = qa.run("新辅助化疗指证")
print(response)

ValueError: Failed with response: <Response [422]>